In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn import svm
from sklearn.model_selection import cross_val_score
from sklearn import metrics
import nltk
nltk.download('stopwords')
import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\bigdata\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
df = pd.read_csv("./scenarii.csv", encoding="ISO-8859-1", sep=";", header=0)
df.head(20)

,texte,tag
0,4.\tVersion Desktop - liste des cas d'utilisat...,texte
1,4.1.1\tCU001-001 Afficher le header,usecase
2,Acteur : Internaute,acteur
3,Cas dutilisation permettant à un internaute ...,texte
4,\tNotes :\nLe système affiche les éléments su...,texte
5,\tActions :\nSi lutilisateur clique\nsur le ...,action
6,4.1.2\tCU001-002 Afficher le bouton de langue,usecase
7,Acteur : Internaute,acteur
8,Cas dutilisation permettant à un internaute ...,texte
9,\tNotes :\nLe système affiche :\nla langue sé...,texte


In [3]:
#Méthode qui vérifie qu'une chaine commence par un entier
def begin_num(texte):
    lis = ['0','1','2','3','4','5','6','7','8','9']
    trouvé = 0
    letter = texte[0]
    for i in lis:
        if letter == i:
            trouvé = 1
    return trouvé

In [4]:
z='9 oranges'
begin_num(z)

1

In [5]:
s='je me nomme cedric'
begin_num(s)


0

In [5]:
from scipy import *

#Recupération des index des cas d'utilisation, action, Notes et Règles de gestion

i=0
useCaseTab=0
actionNoteTab=0
for x in df['tag']:
    str = df.texte[i]
    if x == 'usecase' or (x == 'texte' and begin_num(str)):
        a=i
        useCaseTab=append(useCaseTab,a)
    if x == 'action' or x == 'texte' or x == 'gestion':
        b=i
        actionNoteTab=append(actionNoteTab,b)
   
    i=i+1
print(useCaseTab) # Imprime la valeur courante de la colonne A de df

[   0    0    1    6   12   16   20   26   32   38   41   45   50   56
   60   66   70   76   80   85   90   91   93   98  104  110  120  124
  129  135  136  141  144  148  152  158  162  165  174  175  181  183
  189  191  196  202  208  215  221  226  231  236  244  252  259  267
  272  278  280  281  282  287  288  292  293  296  301  306  311  316
  321  326  332  339  342  344  350  354  358  360  363  367  371  373
  377  378  379  380  385  386  393  402  409  410  415  420  426  432
  438  444  450  454  455  457  458  464  466  471  479  483  488  494
  496  501  509  515  519  525  531  533  538  546  548  553  561  563
  568  574  585  596  603  613  618  623  633  635  637  645  652  657
  659  661  662  670  675  680  687  691  696  702  704  709  711  713
  720  723  725  731  735  740  743  747  752  756  758  762  767  769
  774  776  784  793  798  803  804  809  812  815  820  821  824  831
  839  851  852  856  861  867  872  879  884  889  890  896  900  904
  905 

In [6]:
actionNoteTab

array([   0,    0,    3, ..., 2378, 2380, 2381])

In [7]:
len(actionNoteTab)

1802

In [8]:
len(useCaseTab)

498

## Récuperation des uses cases

In [10]:
##### Récupération des cas d'utilisation
c = "\n"  #séparateur de paragraphes
my_list_len = len(useCaseTab)
matriceCouverture=[]
for i in range(1, my_list_len):
    index = useCaseTab[i]
    texteUC = df["texte"][index]
    liste_ligne = texteUC.split(c)
    matriceCouvertureLigne= liste_ligne[0]
    if matriceCouvertureLigne not in matriceCouverture:
        matriceCouverture = append(matriceCouverture, matriceCouvertureLigne)

In [11]:
matriceCouverture

array(["4.\tVersion Desktop - liste des cas d'utilisations",
       '4.1.1\tCU001-001 Afficher le header',
       '4.1.2\tCU001-002 Afficher le bouton de langue',
       '4.1.3\tCU001-0021 Afficher la page Langue',
       '4.1.4\tCU001-0022 Contribuer la page langue',
       '4.1.5\tCU001-003 Afficher le fil d\x92ariane',
       '4.1.6\tCU001-004 Afficher le bouton de partage',
       '4.1.7\tCU001-005 Afficher le menu de navigation',
       '4.1.8\tCU001-006 Modifier le menu de navigation',
       '4.1.9\tCU001-007 Afficher du contenu dans une autre langue',
       '4.1.10\tCU001-008 Contribuer du contenu dans une autre langue',
       '4.1.11\tCU001-009 Afficher la télécommande',
       '4.1.12\tCU001-010 Modifier la télécommande',
       '4.1.13\tCU001-011 Afficher le footer',
       '4.1.14\tCU001-012 Modifier le footer',
       '4.1.15\tCU001-013 Afficher le bloc de contenu En ce moment',
       '4.1.16\tCU001-014 Modifier le bloc de contenu En ce moment',
       '4.1.17\tCU001-01

In [12]:
len(matriceCouverture)

497

## Récuperation des exigences fonctionnelles

In [13]:
from itertools import chain
from nltk.corpus import wordnet

In [14]:
#Recupération des synonymes en anglais
synonyms = wordnet.synsets('change')
a=set(chain.from_iterable([Word.lemma_names() for Word in synonyms]))
a

{'alter',
 'alteration',
 'change',
 'commute',
 'convert',
 'deepen',
 'exchange',
 'interchange',
 'modification',
 'modify',
 'shift',
 'switch',
 'transfer',
 'variety',
 'vary'}

In [15]:
#Recupération des synonymes en anglais
synonyms = wordnet.synsets('ouvrir')
a=set(chain.from_iterable([Word.lemma_names() for Word in synonyms]))
a

set()

In [16]:
#Recupération des synonymes en français
synonyms = wordnet.synsets('changer', lang=u'fra')
a=set(chain.from_iterable([Word.lemma_names('fra') for Word in synonyms]))
a

{'adapter',
 'bouger',
 'castrer',
 'change',
 'changement',
 'changer',
 'changer_de_main',
 'châtrer',
 'convertir',
 'interrupteur',
 'modification',
 'modifier',
 'monnaie',
 'mutation',
 'passer',
 'pièce_de_monnaie',
 'remplacer',
 'remédier',
 'revoir',
 'traduire',
 'transformer',
 'varier',
 'variété',
 'échange',
 'échanger',
 'échangeur',
 'évolution'}

In [17]:
#Recupération des synonymes en français
synonyms = wordnet.synsets('ouvrir', lang=u'fra')
a=set(chain.from_iterable([Word.lemma_names('fra') for Word in synonyms]))
a

{'activer',
 'air',
 'allumer',
 'amorcer',
 'brancher',
 'commencer',
 'comprendre',
 'donner',
 'débuter',
 'découvert',
 'démarrer',
 'départ',
 'engager',
 'entamer',
 'entreprendre',
 'faire',
 'fermer',
 'flagrant',
 'gagner',
 'libre',
 'mériter',
 'nature',
 'obtenir',
 'offrir',
 'ouvert',
 'ouverture',
 'ouvrir',
 'pionnier',
 'pouvoir',
 'public',
 'recommencer',
 'rendre',
 'réaliser',
 'sujet',
 'superficie',
 'surface',
 'tourner',
 'étendre'}

In [18]:
#Recupération des synonymes en français
synonyms = wordnet.synsets('système', lang=u'fra')
a=set(chain.from_iterable([Word.lemma_names('fra') for Word in synonyms]))
a

{'appareil',
 'cadre',
 'dispositif',
 'mécanisme',
 'organisation',
 'régime',
 'réseau',
 'système'}

In [19]:
for w in a:
    print(w)

appareil
organisation
cadre
mécanisme
système
réseau
régime
dispositif


In [20]:
#Recupération des synonymes en français
synonyms = wordnet.synsets('afficher', lang=u'fra')
a=set(chain.from_iterable([Word.lemma_names('fra') for Word in synonyms]))
a

{'afficher',
 'affirmer',
 'annoncer',
 'dire',
 'déclarer',
 'décréter',
 'exposer',
 'indiquer',
 'manifester',
 'montrer',
 'proclamer',
 'prononcer'}

In [21]:
#Méthode qui vérifie qu'une chaine contient l'une des sous chaines de la liste ou ses synonymes
def is_in(texte, liste):
    trouvé = 0
    for word in liste:
        if word.lower() in texte.lower():
            trouvé = 1
        # Vérification des synonymes
        synonyms = wordnet.synsets(word, lang=u'fra')
        l=set(chain.from_iterable([Word.lemma_names('fra') for Word in synonyms]))
        for word in l:
            if word.lower() in texte.lower():
                trouvé = 1
    return trouvé

In [22]:
l=["fermer"]
a = is_in("ouvrir un restaurant n'est pas mal", l)

In [23]:
a

1

In [24]:
#Liste des mots décrivant des exigences metiers
liste = ['affiche le', 'affiché', 'ouvrir', 'mettre en surbrillance ', 'fenêtre s’ouvre ', 'cliquer', 'sélectionner', 'clic', 'utilisateur peut', 'contributeur peut', 'peut créer', 'système redirige', 'système affiche', 'se place', 'est composé', 'cliquable', 'sélectionne', 'a sélectionné', 'acceder', 'permet à', 'rediriger','permettre']

In [73]:
#Récuperation des exigences fonctionnelles
c = "\n"  #séparateur de paragraphes
s1 = "suivants :"
s3 = "suivante :"
#s2 = "affiche :"
vec = np.zeros((len(useCaseTab),), dtype=int)
my_list_len_actionTab = len(actionNoteTab)
matriceCouverture1=[]
matriceCouverture2=[]
i = 3
k = 0
indexi = useCaseTab[i]
for j in range(2, my_list_len_actionTab):
    indexj = actionNoteTab[j]
    if (indexj >= indexi and i < len(matriceCouverture)-1):
        i = i+1
        k = k+1
        indexi = useCaseTab[i]
    if (indexj>=useCaseTab[len(matriceCouverture)]) and (k < len(matriceCouverture)):
        k = k+1
    chaine = df["texte"][indexj]
    for x in chaine.split(c):
        val = is_in(x, liste)
        if val == 1 and (not begin_num(x)) and (not x.endswith(s1)) and (not x.endswith(s3)):
            if x not in matriceCouverture1:
                matriceCouverture1 = append(matriceCouverture1, x)
            matriceCouverture2 = append(matriceCouverture2, x)
            vec[k] = vec[k]+1
    #if vec[k]==0:
        

In [74]:
k

496

In [75]:
vec

array([ 5, 10,  1,  2,  2,  4,  7,  3,  3,  3, 10,  1,  2,  3,  5,  2,  6,
        2,  0,  0,  2,  4,  9, 14,  1,  3, 10,  0,  4,  2,  2,  3,  4,  2,
        2,  9,  0,  6,  0,  6,  0,  3,  4,  4,  1,  4,  2,  2,  3,  7,  6,
        9, 15,  1,  5,  0,  0,  0,  1,  0,  3,  0,  2,  4,  7,  4,  8,  5,
        7,  5,  6,  2,  1,  5,  3,  3,  3,  3,  2,  5,  1,  5,  0,  0,  0,
        2,  0,  3,  2,  8,  0,  1,  7,  7,  8,  5,  7,  5,  0,  0,  0,  0,
        6,  0,  1, 20,  1,  5,  3,  0,  1,  8,  7,  1,  4,  3,  0,  1, 17,
        1,  1, 11,  0,  4,  6, 18,  4,  1,  6,  2,  3,  8,  1,  1,  7,  9,
        1,  1,  0,  0, 12,  4,  1,  8,  2,  5,  3,  0,  4,  0,  0,  6,  2,
        1,  5,  3,  8,  3,  6,  2,  4,  1,  8,  3,  0,  1,  1, 12,  5,  8,
        4,  0,  2,  2,  6,  6,  0,  2, 19, 36, 17,  0,  4,  2, 11,  4,  8,
        2,  4,  0,  4,  3,  7,  0,  2,  4,  7,  0, 28,  1,  0,  2,  5,  6,
        0, 16,  1,  0,  3,  4,  8,  0,  7,  0,  2,  3,  9,  0, 14,  1,  0,
        2,  5,  5,  0, 18

In [76]:
len(vec)

498

In [77]:
matriceCouverture

array(["4.\tVersion Desktop - liste des cas d'utilisations",
       '4.1.1\tCU001-001 Afficher le header',
       '4.1.2\tCU001-002 Afficher le bouton de langue',
       '4.1.3\tCU001-0021 Afficher la page Langue',
       '4.1.4\tCU001-0022 Contribuer la page langue',
       '4.1.5\tCU001-003 Afficher le fil d\x92ariane',
       '4.1.6\tCU001-004 Afficher le bouton de partage',
       '4.1.7\tCU001-005 Afficher le menu de navigation',
       '4.1.8\tCU001-006 Modifier le menu de navigation',
       '4.1.9\tCU001-007 Afficher du contenu dans une autre langue',
       '4.1.10\tCU001-008 Contribuer du contenu dans une autre langue',
       '4.1.11\tCU001-009 Afficher la télécommande',
       '4.1.12\tCU001-010 Modifier la télécommande',
       '4.1.13\tCU001-011 Afficher le footer',
       '4.1.14\tCU001-012 Modifier le footer',
       '4.1.15\tCU001-013 Afficher le bloc de contenu En ce moment',
       '4.1.16\tCU001-014 Modifier le bloc de contenu En ce moment',
       '4.1.17\tCU001-01

In [78]:
for i in range(0,len(vec)-2):
    if vec[i]==0:
        print("\n",matriceCouverture[i+1])


 4.2	Page dAccueil [LOT 1]

 4.2.1	Afficher la page daccueil [LOT 1]

 4.2.2	Contribuer la page daccueil [LOT 1]

 4.3	Page dEditoriale (LOT 2)

 4.3.2	Afficher une liste spécifique de pages éditoriales et / ou dévénements

 4.3.3	Afficher une page éditoriale

 4.3.3.p	CU003-015 Afficher un bloc Plan interactif

 4.3.3.a	CU003-0155 Afficher un bloc Parcours [LOT 4]

 4.3.4	Afficher une page éditoriale importée

 4.3.5	Contribuer une liste spécifique de pages éditoriales et / ou dévénement

 4.3.6	Contribuer une page éditoriale

 4.4	Collections [LOT 3]

 4.4.1	CU004-001 Afficher les collections sur la page daccueil

 4.5	Recherche [LOT 3]

 4.5.2	Afficher la recherche

 4.5.3	Afficher la liste des résultats

 4.5.3.h	CU005-010 Afficher le détail dun élément résultat

 4.6	Evènement [LOT 2]

 4.6.1	Afficher une liste spécifique de pages éditoriales et / ou dévénements

 4.6.2	Afficher les catégories dévénements  [LOT 2]

 4.6.3	Afficher les événements liés à une ou plusieurs 

In [79]:
print(matriceCouverture1)

['le système affiche les boutons des réseaux sociaux suivants'
 'sur le logo depuis une page intérieure, le système redirige vers la page d\x92accueil du site sans jouer la vidéo du bloc identitaire et sans les informations temps réels..'
 'Sur le logo depuis la page d\x92accueil, le système affiche le bloc d\x92information temps réel et le bloc identitaire'
 ... 'Les ronds ne sont pas cliquables'
 'Au clic sur le visuel du parcours, le système redirige vers le parcours concerné'
 'Libellé Champ de partage twitter    + Titre + lien raccourci de la page de démarrage du parcours + Champ libre twiitter']


In [80]:
matriceCouverture1.shape

(1177,)

In [81]:
matriceCouverture2.shape

(2360,)

In [82]:
import pickle

with open("test.txt", "wb") as fp:   #Pickling
    pickle.dump(matriceCouverture1, fp)

#with open("test.txt", "rb") as fp:   # Unpickling
   #b = pickle.load(fp)

In [83]:
for s in matriceCouverture1:
    print(s,'\n')

le système affiche les boutons des réseaux sociaux suivants 

sur le logo depuis une page intérieure, le système redirige vers la page daccueil du site sans jouer la vidéo du bloc identitaire et sans les informations temps réels.. 

Sur le logo depuis la page daccueil, le système affiche le bloc dinformation temps réel et le bloc identitaire 

Si lutilisateur clique sur le bouton daccessibilité, le système redirige vers une page éditoriale. 

Un clic sur un bouton de réseaux sociaux permet à lutilisateur daccéder au réseau social concerné (dans une nouvelle fenêtre) : 

Le système affiche : 

Au clic sur la langue sélectionnée, le système affiche les autres langues 

Si la langue sélectionnée est français, le système affiche 

Si la langue sélectionnée est anglais, le système affiche 

Si la langue sélectionnée est espagnol, le système affiche 

Au clic sur la deuxième ou troisième langue, le système réaffiche  les boutons réseaux sociaux et le bouton + est caché 

Si la page es

In [84]:
import pandas
l = matriceCouverture.transpose()
df_usecase = pandas.DataFrame(l)
df_usecase.head(10)

,0
0,4.\tVersion Desktop - liste des cas d'utilisat...
1,4.1.1\tCU001-001 Afficher le header
2,4.1.2\tCU001-002 Afficher le bouton de langue
3,4.1.3\tCU001-0021 Afficher la page Langue
4,4.1.4\tCU001-0022 Contribuer la page langue
5,4.1.5\tCU001-003 Afficher le fil dariane
6,4.1.6\tCU001-004 Afficher le bouton de partage
7,4.1.7\tCU001-005 Afficher le menu de navigation
8,4.1.8\tCU001-006 Modifier le menu de navigation
9,4.1.9\tCU001-007 Afficher du contenu dans une ...


In [85]:
import pandas
l1 =  [matriceCouverture2]
df_EM = pandas.DataFrame(l1)
df_EM

,0,1,2,3,4,5,6,7,8,9,...,2350,2351,2352,2353,2354,2355,2356,2357,2358,2359
0,le système affiche les boutons des réseaux soc...,"sur le logo depuis une page intérieure, le sys...","Sur le logo depuis la page daccueil, le systè...",Si lutilisateur clique sur le bouton daccess...,Un clic sur un bouton de réseaux sociaux perme...,Le système affiche :,"Au clic sur la langue sélectionnée, le système...","Si la langue sélectionnée est français, le sys...","Si la langue sélectionnée est anglais, le syst...","Si la langue sélectionnée est espagnol, le sys...",...,Si lutilisateur a sélectionné une langue angl...,Le système affiche 2 menus déroulants,\tLe système affiche les libellés suivants,La liste des catégories de parcours affichées ...,"\tPar défaut, le système affiche Tous les par...",\tLe système affiche les libellés suivants,"\tPar défaut, le système affiche les plus réc...",Si lutilisateur a sélectionné une langue angl...,Le système affiche la liste des parcours,"Pour chaque parcours, le système affiche"


## Construction de la matrice

In [86]:

import pandas
l = matriceCouverture.transpose()
a1 = len(l)
a2 = len(matriceCouverture1)
a3=a2-1
A = np.empty((a1,a2), dtype='<U200')

mc = pandas.DataFrame(A, index = l, columns = matriceCouverture1)

#C = np.c_[l,A]
#mc = pandas.DataFrame(C)
#mc.columns = [matriceCouverture1]
mc

,le système affiche les boutons des réseaux sociaux suivants,"sur le logo depuis une page intérieure, le système redirige vers la page daccueil du site sans jouer la vidéo du bloc identitaire et sans les informations temps réels..","Sur le logo depuis la page daccueil, le système affiche le bloc dinformation temps réel et le bloc identitaire","Si lutilisateur clique sur le bouton daccessibilité, le système redirige vers une page éditoriale.",Un clic sur un bouton de réseaux sociaux permet à lutilisateur daccéder au réseau social concerné (dans une nouvelle fenêtre) :,Le système affiche :,"Au clic sur la langue sélectionnée, le système affiche les autres langues","Si la langue sélectionnée est français, le système affiche","Si la langue sélectionnée est anglais, le système affiche","Si la langue sélectionnée est espagnol, le système affiche",...,Déplacement du doigt de gauche à droite : le système affiche lélément suivant,Déplacement du doigt de droite à gauche : le système affiche lélément précédent,Au clic sur un élément du parcours,Le libellé Aller à luvre suivante,"Au clic sur la croix, on réinitialise laffichage de lélément sans zoom","Au clic sur le libellé Aller à luvre suivante, le système affiche lélément suivant","Les blocs de texte sont affichés à la suite, sous le visuel",Les ronds ne sont pas cliquables,"Au clic sur le visuel du parcours, le système redirige vers le parcours concerné",Libellé Champ de partage twitter + Titre + lien raccourci de la page de démarrage du parcours + Champ libre twiitter
4.\tVersion Desktop - liste des cas d'utilisations,,,,,,,,,,,...,,,,,,,,,,
4.1.1\tCU001-001 Afficher le header,,,,,,,,,,,...,,,,,,,,,,
4.1.2\tCU001-002 Afficher le bouton de langue,,,,,,,,,,,...,,,,,,,,,,
4.1.3\tCU001-0021 Afficher la page Langue,,,,,,,,,,,...,,,,,,,,,,
4.1.4\tCU001-0022 Contribuer la page langue,,,,,,,,,,,...,,,,,,,,,,
4.1.5\tCU001-003 Afficher le fil dariane,,,,,,,,,,,...,,,,,,,,,,
4.1.6\tCU001-004 Afficher le bouton de partage,,,,,,,,,,,...,,,,,,,,,,
4.1.7\tCU001-005 Afficher le menu de navigation,,,,,,,,,,,...,,,,,,,,,,
4.1.8\tCU001-006 Modifier le menu de navigation,,,,,,,,,,,...,,,,,,,,,,
4.1.9\tCU001-007 Afficher du contenu dans une autre langue,,,,,,,,,,,...,,,,,,,,,,


In [87]:
#mc['Sur le logo depuis la page daccueil, le système affiche le bloc dinformation temps réel et le bloc identitaire']["5.8.7.d\tCU009-4025 Afficher le document dans le bloc principal"]

## Insertion des liens directs

In [88]:
id = 0
som = 0
for index_cu in range(0, len(vec)-1):
    nb_em = vec[index_cu]
    som = som + nb_em
    if nb_em > 0:
        for index_em in range(id, som):
            uc = df_usecase[0][index_cu]
            em = df_EM[index_em]
            mc.at[uc,em] = 'X'
        id = id + nb_em
        

In [89]:
mc

,le système affiche les boutons des réseaux sociaux suivants,"sur le logo depuis une page intérieure, le système redirige vers la page daccueil du site sans jouer la vidéo du bloc identitaire et sans les informations temps réels..","Sur le logo depuis la page daccueil, le système affiche le bloc dinformation temps réel et le bloc identitaire","Si lutilisateur clique sur le bouton daccessibilité, le système redirige vers une page éditoriale.",Un clic sur un bouton de réseaux sociaux permet à lutilisateur daccéder au réseau social concerné (dans une nouvelle fenêtre) :,Le système affiche :,"Au clic sur la langue sélectionnée, le système affiche les autres langues","Si la langue sélectionnée est français, le système affiche","Si la langue sélectionnée est anglais, le système affiche","Si la langue sélectionnée est espagnol, le système affiche",...,Déplacement du doigt de gauche à droite : le système affiche lélément suivant,Déplacement du doigt de droite à gauche : le système affiche lélément précédent,Au clic sur un élément du parcours,Le libellé Aller à luvre suivante,"Au clic sur la croix, on réinitialise laffichage de lélément sans zoom","Au clic sur le libellé Aller à luvre suivante, le système affiche lélément suivant","Les blocs de texte sont affichés à la suite, sous le visuel",Les ronds ne sont pas cliquables,"Au clic sur le visuel du parcours, le système redirige vers le parcours concerné",Libellé Champ de partage twitter + Titre + lien raccourci de la page de démarrage du parcours + Champ libre twiitter
4.\tVersion Desktop - liste des cas d'utilisations,X,X,X,X,X,,,,,,...,,,,,,,,,,
4.1.1\tCU001-001 Afficher le header,,,,,,X,X,X,X,X,...,,,,,,,,,,
4.1.2\tCU001-002 Afficher le bouton de langue,,,,,,,,,,,...,,,,,,,,,,
4.1.3\tCU001-0021 Afficher la page Langue,,,,,,,,,,,...,,,,,,,,,,
4.1.4\tCU001-0022 Contribuer la page langue,,,,,,,,,,,...,,,,,,,,,,
4.1.5\tCU001-003 Afficher le fil dariane,,,,,,,,,,,...,,,,,,,,,,
4.1.6\tCU001-004 Afficher le bouton de partage,,,,,,,,,,,...,,,,,,,,,,
4.1.7\tCU001-005 Afficher le menu de navigation,,,,,,,,,,,...,,,,,,,,,,
4.1.8\tCU001-006 Modifier le menu de navigation,,,,,,,,,,,...,,,,,,,,,,
4.1.9\tCU001-007 Afficher du contenu dans une autre langue,,,,,,,,,,,...,,,,,,,,,,


# DETERMINATION DES LIENS INDIRECTS A PARTIR DU KNN

## Construction du modèle KNN

In [90]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

In [91]:
#Mis en forme des données (Exigences Fonctionnelles) pour la construction du modèle
mt = matriceCouverture1.transpose()
data = pandas.DataFrame(mt)
data.head(10)

,0
0,le système affiche les boutons des réseaux soc...
1,"sur le logo depuis une page intérieure, le sys..."
2,"Sur le logo depuis la page daccueil, le systè..."
3,Si lutilisateur clique sur le bouton daccess...
4,Un clic sur un bouton de réseaux sociaux perme...
5,Le système affiche :
6,"Au clic sur la langue sélectionnée, le système..."
7,"Si la langue sélectionnée est français, le sys..."
8,"Si la langue sélectionnée est anglais, le syst..."
9,"Si la langue sélectionnée est espagnol, le sys..."


In [92]:
data.columns = ['texte']
data.head(10)

,texte
0,le système affiche les boutons des réseaux soc...
1,"sur le logo depuis une page intérieure, le sys..."
2,"Sur le logo depuis la page daccueil, le systè..."
3,Si lutilisateur clique sur le bouton daccess...
4,Un clic sur un bouton de réseaux sociaux perme...
5,Le système affiche :
6,"Au clic sur la langue sélectionnée, le système..."
7,"Si la langue sélectionnée est français, le sys..."
8,"Si la langue sélectionnée est anglais, le syst..."
9,"Si la langue sélectionnée est espagnol, le sys..."


In [93]:
vec = TfidfVectorizer(max_features=500, stop_words=stopwords.words('french'))
vec.fit(data['texte'].values)
features = vec.transform(data['texte'].values)

In [94]:
features

<1177x500 sparse matrix of type '<class 'numpy.float64'>'
	with 8093 stored elements in Compressed Sparse Row format>

In [95]:
features.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [96]:
#La Métrique cosine est généralement utilisé pour la similitude de texte
knn = NearestNeighbors(n_neighbors=10, metric='cosine')
knn.fit(features)

NearestNeighbors(algorithm='auto', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=None, n_neighbors=10, p=2,
                 radius=1.0)

In [97]:
#Observation des 10 voisins les plus proches au 1er texte ainsi que les distances qui les séparent
knn.kneighbors(features[0:1], return_distance=True)

(array([[0.        , 0.19586056, 0.56338204, 0.6362111 , 0.6362111 ,
         0.65619372, 0.65619372, 0.6737987 , 0.70249331, 0.72694139]]),
 array([[   0,   10,  837,   52,    4, 1053,  825,   20, 1111,  435]],
       dtype=int64))

In [98]:
df_usecase.head(10)

,0
0,4.\tVersion Desktop - liste des cas d'utilisat...
1,4.1.1\tCU001-001 Afficher le header
2,4.1.2\tCU001-002 Afficher le bouton de langue
3,4.1.3\tCU001-0021 Afficher la page Langue
4,4.1.4\tCU001-0022 Contribuer la page langue
5,4.1.5\tCU001-003 Afficher le fil dariane
6,4.1.6\tCU001-004 Afficher le bouton de partage
7,4.1.7\tCU001-005 Afficher le menu de navigation
8,4.1.8\tCU001-006 Modifier le menu de navigation
9,4.1.9\tCU001-007 Afficher du contenu dans une ...


### Affichons les exigences metiers les plus proches de 20 premiers use case

In [99]:

for usecase in df_usecase[0][:20]:
    print("Use Case = ", usecase, "\n")
    input_features = vec.transform([usecase])
    D, N = knn.kneighbors(input_features, n_neighbors=10, return_distance=True)
    for input_text, distances, neighbors in zip(usecase, D, N):
        #print("Input text = ", input_text[:200], "\n")
        for dist, neighbor_idx in zip(distances, neighbors):
            if dist < 0.5 :
                print("Distance = ", dist, "Neighbor idx = ", neighbor_idx)
                print(data['texte'].values[neighbor_idx][:])
                print("-"*200)
    print("="*200)
    print('\n\n')

Use Case =  4.	Version Desktop - liste des cas d'utilisations 




Use Case =  4.1.1	CU001-001 Afficher le header 




Use Case =  4.1.2	CU001-002 Afficher le bouton de langue 

Distance =  0.29752536234298543 Neighbor idx =  1007
Le système affiche le bouton de langue
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Distance =  0.31349321363413707 Neighbor idx =  1005
	le bouton de langue (voir cas dutilisation affiche le bouton de langue)
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Distance =  0.4491252324986539 Neighbor idx =  1008
Au clic sur le bouton de la langue, le système affiche une liste de langue avec pour chaque langue ci-dessous le libellé de la langue traduite et un

## INSERTION DES LIENS INDIRECT

In [100]:

for usecase in df_usecase[0]:
    #print("Input text = ", usecase, "\n")
    input_features = vec.transform([usecase])
    D, N = knn.kneighbors(input_features, n_neighbors=10, return_distance=True)
    for input_text, distances, neighbors in zip(usecase, D, N):
        #print("Input text = ", input_text[:200], "\n")
        for dist, neighbor_idx in zip(distances, neighbors):
            exi_met = data['texte'].values[neighbor_idx]
            if dist < 0.7 :
                if mc[exi_met][usecase] != 'X':
                    mc.ix[usecase.strip(),exi_met.strip()] = 'O'

In [101]:
mc

,le système affiche les boutons des réseaux sociaux suivants,"sur le logo depuis une page intérieure, le système redirige vers la page daccueil du site sans jouer la vidéo du bloc identitaire et sans les informations temps réels..","Sur le logo depuis la page daccueil, le système affiche le bloc dinformation temps réel et le bloc identitaire","Si lutilisateur clique sur le bouton daccessibilité, le système redirige vers une page éditoriale.",Un clic sur un bouton de réseaux sociaux permet à lutilisateur daccéder au réseau social concerné (dans une nouvelle fenêtre) :,Le système affiche :,"Au clic sur la langue sélectionnée, le système affiche les autres langues","Si la langue sélectionnée est français, le système affiche","Si la langue sélectionnée est anglais, le système affiche","Si la langue sélectionnée est espagnol, le système affiche",...,Déplacement du doigt de gauche à droite : le système affiche lélément suivant,Déplacement du doigt de droite à gauche : le système affiche lélément précédent,Au clic sur un élément du parcours,Le libellé Aller à luvre suivante,"Au clic sur la croix, on réinitialise laffichage de lélément sans zoom","Au clic sur le libellé Aller à luvre suivante, le système affiche lélément suivant","Les blocs de texte sont affichés à la suite, sous le visuel",Les ronds ne sont pas cliquables,"Au clic sur le visuel du parcours, le système redirige vers le parcours concerné",Libellé Champ de partage twitter + Titre + lien raccourci de la page de démarrage du parcours + Champ libre twiitter
4.\tVersion Desktop - liste des cas d'utilisations,X,X,X,X,X,,,,,,...,,,,,,,,,,
4.1.1\tCU001-001 Afficher le header,,,,,,X,X,X,X,X,...,,,,,,,,,,
4.1.2\tCU001-002 Afficher le bouton de langue,,,,,,,,,,,...,,,,,,,,,,
4.1.3\tCU001-0021 Afficher la page Langue,,,,,,,,,,,...,,,,,,,,,,
4.1.4\tCU001-0022 Contribuer la page langue,,,,,,,,,,,...,,,,,,,,,,
4.1.5\tCU001-003 Afficher le fil dariane,,,,,,,,,,,...,,,,,,,,,,
4.1.6\tCU001-004 Afficher le bouton de partage,,,,,,,,,,,...,,,,,,,,,,
4.1.7\tCU001-005 Afficher le menu de navigation,,,,,,,,,,,...,,,,,,,,,,
4.1.8\tCU001-006 Modifier le menu de navigation,,,,,,,,,,,...,,,,,,,,,,
4.1.9\tCU001-007 Afficher du contenu dans une autre langue,,,,,,,,,,,...,,,,,,,,,,


In [66]:
#df.to_csv (r'Path où vous souhaitez stocker le fichier CSV exporté \ File Name.csv ')

In [67]:
mc.to_excel("C:/Users/bigdata/Documents/stage Altran/code/clustering/Matric_Couverture2.xlsx") 